In [1]:
import logging

import pandas as pd

from qlearning.envs.trading_env import TradingEnv
from qlearning.agents.dqn_agent import DQNAgent
from learning.preprocessing import reshape_with_window
from sklearn.model_selection import train_test_split

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
log_fh = logging.FileHandler('logs/dqn_lstm.log')
logger.addHandler(log_fh)

In [14]:
def get_data(reshape3d=True):
    df = pd.read_csv('data/gbp_cad_finam.csv', index_col=['time'])

    reshape_columns = [c for c in df.columns if 'finam' in c]
    new_df = reshape_with_window(df, [1,2,3,4,5,6,7,8,9,10], reshape_columns, reshape3d)
    new_df = new_df[-1500:]
    X = new_df[-1500:]
    if reshape3d:
        Y = X[:,-1,0]
    else:
        Y = X[:,0]
    return X, Y

In [15]:
X_data, Y_data = get_data(reshape3d=True)
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, shuffle=False)

In [18]:
train_env = TradingEnv(X_train, Y_train, order_close_threshold=0.015, order_tax=0.005)
state_size = train_env.observation_space.shape[1]
action_size = train_env.action_space.n

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

# model = Sequential()
# model.add(Dense(10, input_dim=state_size, activation='relu'))
# for _ in range(10):
#     model.add(Dense(10, activation='relu'))
# model.add(Dense(action_size, activation='softmax'))
# model.compile(loss='mse', optimizer=Adam())

def create_model(df):
    model = Sequential()
    model.add(LSTM(20, input_shape=df.shape[1:]))
    model.add(Dropout(0.1))
    model.add(Flatten(input_shape=df.shape[1:]))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(action_size, activation='softmax'))
    model.compile(loss='mse', optimizer='Adam', metrics=['accuracy'])
    
    return model

model = create_model(X_train)

W0909 17:36:28.738510 139967994881856 deprecation.py:506] From /usr/local/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
agent = DQNAgent(model=model, action_size=action_size)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20)                3200      
_________________________________________________________________
dropout (Dropout)            (None, 20)                0         
_________________________________________________________________
flatten (Flatten)            (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                210       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0

In [11]:
EPISODES = 10
BATCH_SIZE = 32

for e in range(EPISODES):
    state = train_env.reset()
    actions = [0,0,0]
    for step in range(train_env.n_step):
        action = agent.act(state)
        actions[action] += 1
        next_state, reward, done, info = train_env.step(action)
        agent.remember(state, action, reward, next_state, done)
        
        if done:
            logger.info(f'episode: {e+1}/{EPISODES}, end value: {train_env.wallet_amount}. actions: {actions}')
        if len(agent.memory) > BATCH_SIZE:
            agent.replay(BATCH_SIZE)

KeyboardInterrupt: 